In [8]:
from tensorflow import keras
import os
import numpy as np
import cv2

In [9]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

print(len(x_train))

# Define the path to the folder containing the image folders
base_path = "printed_digits_dataset/assets/"

# Initialize the x and y variables
x = []
y = []

# Loop through the folders labeled 1-10
for label in range(0, 10):
    folder_path = os.path.join(base_path, str(label))
    
    # Get the list of image files in the folder
    image_files = os.listdir(folder_path)
    
    # Loop through each image file in the folder
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        
        # Load the image using OpenCV
        image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        # Append the image to the x variable
        x.append(image)
        
        # Append the label to the y variable
        y.append(label)

# Convert the x variable to a numpy array
x_train = np.concatenate((x_train,np.array(x)),axis=0)

# Convert the y variable to a numpy array
y_train = np.concatenate((y_train,np.array(y)),axis=0)

print(len(x_train))

# Preprocess the data (e.g., normalize pixel values)
x_train = x_train / 255.0
x_test = x_test / 255.0

# Define the model architecture
mnist_model2 = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
mnist_model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape the data to match the input shape of the new model
x_train2 = np.expand_dims(x_train, axis=-1)
x_test2 = np.expand_dims(x_test, axis=-1)

from keras.preprocessing.image import ImageDataGenerator

# Augment data a little
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train2)

# Train the model
mnist_model2.fit(datagen.flow(x_train2, y_train, batch_size=32),
                epochs=10, validation_data=(x_test2, y_test))

60000
66299
Epoch 1/10
2072/2072 [==============================] - 15s 7ms/step - loss: 0.4704 - accuracy: 0.8527 - val_loss: 0.0418 - val_accuracy: 0.9856
Epoch 2/10
2072/2072 [==============================] - 15s 7ms/step - loss: 0.2032 - accuracy: 0.9367 - val_loss: 0.0347 - val_accuracy: 0.9879
Epoch 3/10
2072/2072 [==============================] - 15s 7ms/step - loss: 0.1619 - accuracy: 0.9504 - val_loss: 0.0262 - val_accuracy: 0.9916
Epoch 4/10
2072/2072 [==============================] - 15s 7ms/step - loss: 0.1336 - accuracy: 0.9595 - val_loss: 0.0317 - val_accuracy: 0.9905
Epoch 5/10
2072/2072 [==============================] - 15s 7ms/step - loss: 0.1206 - accuracy: 0.9635 - val_loss: 0.0212 - val_accuracy: 0.9924
Epoch 6/10
2072/2072 [==============================] - 15s 7ms/step - loss: 0.1094 - accuracy: 0.9675 - val_loss: 0.0209 - val_accuracy: 0.9929
Epoch 7/10
2072/2072 [==============================] - 15s 7ms/step - loss: 0.1011 - accuracy: 0.9702 - val_loss: 0.0

In [10]:
mnist_model2.save('digit_recognizer.keras')

In [11]:
mnist_model2.save_weights('digit_recognizer.h5')